In [ ]:
import matplotlib.pyplot as plt
import datetime
import tilemapbase as tmb
import numpy as np
import pandas as pd
from pluma.stream.georeference import Georeference
from pluma.stream.ubx import _UBX_MSGIDS
from pluma.schema import Dataset

## Notebook plotting
%matplotlib inline
plt.style.use('ggplot')

## Figure export parameters
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
import matplotlib as mpl
mpl.rcParams.update(new_rc_params)

In [ ]:
stream_root_folder = r's3:///emotional-cities/data/nrg/poc-v1/TechnicalBenchmarks/SoundCardTKbenchmark2022_09_27_15_16_36'
from benchmark_schemas import build_benchmarkschema_microphone

dataset = Dataset(
    stream_root_folder,
    datasetlabel="Microphone_Benchmark",
    georeference= Georeference(),
    schema=build_benchmarkschema_microphone)
dataset.populate_streams(autoload=True)



In [ ]:
mic_waveform = dataset.streams.Microphone.Audio.data[:,1]
mic_waveform_filt = pd.DataFrame(mic_waveform).rolling(10000).max().to_numpy().flatten() #get rid of double hits due to hysteresis using a max-window filter.

threshold = 1000
ttl = (mic_waveform_filt > threshold).astype(int)
ttl_rising_edges_loc = np.where(np.diff(ttl, prepend=np.NaN) == 1)[0]
ttl_rising_edges = mic_waveform_filt[ttl_rising_edges_loc]


plt.figure(figsize=(10, 5))
plt.plot(mic_waveform_filt)
plt.scatter(ttl_rising_edges_loc, ttl_rising_edges, color='green')
plt.show()


In [ ]:
## load the harp-software timestamped buffers
buffers = dataset.streams.Microphone.BufferIndex.data
fs = 44100
buffer_size = 441
print(f"""
Fs = {fs} Hz
Samples = {len(mic_waveform)}
Buffers = {len(buffers)}
Expected buffer size = {buffer_size}
Samples per buffer = {len(mic_waveform)/len(buffers)}
Buffer predicted duration = {len(mic_waveform)/len(buffers)/44100} s
""")

buffer_idx = np.floor(ttl_rising_edges_loc / buffer_size).astype(int).transpose()
buffer_ts = buffers.iloc[buffer_idx,:]
# Get the synch pulse from the BioData "Set" stream
bit_mask = 0x03
rising_edge_events = dataset.streams.BioData.Set.data[dataset.streams.BioData.Set.data & bit_mask > 0]

delta_t = rising_edge_events.index.values - buffer_ts.index.values
delta_t = -delta_t / np.timedelta64(1, 'ms')
delta_t = delta_t[delta_t > 0]

plt.figure(figsize= (9,6))
plt.hist(delta_t, bins = 25, color= "Teal")
plt.xlabel('Latency (ms)')
plt.ylabel('Counts')
plt.xlim((0,100))
plt.title(f'Microphone latency benchmark\n$\mu = {np.mean(delta_t):.2f}, \sigma = {np.std(delta_t):.2f},$ [min:max] = [{np.min(delta_t):.2f}:{np.max(delta_t):.2f}]')
plt.show()